In [3]:
#pip install requests beautifulsoup4

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
def get_soup(url, delay=1, headers=None, proxy=None):
    """
    Fetches and parses a webpage with BeautifulSoup, handling 403 errors
    by setting headers, delays, and optional proxies.

    Parameters:
        url (str): The URL of the webpage to scrape.
        delay (int): The delay (in seconds) between requests. Default is 1 second.
        headers (dict): Optional headers to include in the request.
        proxy (dict): Optional dictionary for proxies.

    Returns:
        BeautifulSoup object of the parsed page or None if request fails.
    """

    # Default headers to mimic a browser request if none are provided
    if headers is None:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }

    try:
        # Send the request with headers and optional proxy
        response = requests.get(url, headers=headers, proxies=proxy)
        
        # Check for 403 Forbidden error
        if response.status_code == 403:
            print("403 Forbidden: Access to the page is restricted.")
            return None
        
        # Add delay to avoid rapid requests
        time.sleep(delay)

        # Parse the content with BeautifulSoup
        soup = BeautifulSoup(response.content, "html.parser")
        return soup
    
    except requests.RequestException as e:
        print(f"An error occurred: {e}")
        return None


In [89]:
def scrape_list(urls, year):
    try:
        # Ensure the year exists in the provided URLs
        if year not in urls:
            print(f"Year {year} not found in the URL dictionary.")
            return None
        
        # Get the URL for the given year
        url = urls[year]

        # Set headers to mimic a browser request
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }
        
        # Send a GET request to the URL
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')

        # Determine which instance of 'tablesorter' to use
        tables = soup.find_all('table', class_='tablesorter')
        if not tables:
            print(f"No tables with class 'tablesorter' found for {url}")
            return None
        
        # Choose the appropriate table based on the year
        if (year <= 2000 and year not in [1998, 1999]) or (year >= 2013 and year <= 2019):
            # Use the first instance of the table
            data_table = tables[0] if len(tables) > 0 else None
        else:
            # Use the second instance of the table
            data_table = tables[1] if len(tables) > 1 else None
        
        if not data_table:
            print(f"Appropriate data table not found for {url} and year {year}")
            return None

        # Extract data from tbody
        tbody = data_table.find('tbody')
        if not tbody:
            print(f"No tbody found in the selected table for {url}")
            return None
            
        standings = []
        for tr in tbody.find_all('tr'):
            tds = tr.find_all('td')
            if len(tds) >= 2:  # Assuming we need at least rank and team
                rank = tds[0].text.strip()
                team = tds[1].text.strip()
                standings.append(f"{rank} – {team}")

        return standings

    except requests.HTTPError as http_err:
        print(f"HTTP error occurred while scraping {year}: {http_err}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred while scraping {year}: {e}")
        return None


In [90]:
def scrape_multiple_urls_dict(urls):
    # Dictionary to store the results for each year
    results = {}
    for year, url in urls.items():
        print(f"Scraping data for the year {year} from {url}...")
        # Pass the urls dictionary and the year to scrape_list
        data = scrape_list(urls, year)
        results[year] = data if data else "No data found"
    return results


In [91]:
urls = {
    1979: 'https://archive.usaultimate.org/archives/1979_club.aspx',
    1980: 'https://archive.usaultimate.org/archives/1980_club.aspx',
    1981: 'https://archive.usaultimate.org/archives/1981_club.aspx',
    1982: 'https://archive.usaultimate.org/archives/1982_club.aspx',
    1983: 'https://archive.usaultimate.org/archives/1983_club.aspx',
    1984: 'https://archive.usaultimate.org/archives/1984_club.aspx',
    1985: 'https://archive.usaultimate.org/archives/1985_club.aspx',
    1986: 'https://archive.usaultimate.org/archives/1986_club.aspx',
    1987: 'https://archive.usaultimate.org/archives/1987_club.aspx',
    1988: 'https://archive.usaultimate.org/archives/1988_club.aspx',
    1989: 'https://archive.usaultimate.org/archives/1989_club.aspx',
    1990: 'https://archive.usaultimate.org/archives/1990_club.aspx',
    1991: 'https://archive.usaultimate.org/archives/1991_club.aspx',
    1992: 'https://archive.usaultimate.org/archives/1992_club.aspx',
    1993: 'https://archive.usaultimate.org/archives/1993_club.aspx',
    1994: 'https://archive.usaultimate.org/archives/1994_club.aspx',
    1995: 'https://archive.usaultimate.org/archives/1995_club.aspx',
    1996: 'https://archive.usaultimate.org/archives/1996_club.aspx',
    1997: 'https://archive.usaultimate.org/archives/1997_club.aspx',
    1998: 'https://archive.usaultimate.org/archives/1998_club.aspx',
    1999: 'https://archive.usaultimate.org/archives/1999_club.aspx',
    2000: 'https://archive.usaultimate.org/archives/2000_club.aspx',
    2001: 'https://archive.usaultimate.org/archives/2001_club.aspx',
    2002: 'https://archive.usaultimate.org/archives/2002_club.aspx',
    2003: 'https://archive.usaultimate.org/archives/2003_club.aspx',
    2004: 'https://archive.usaultimate.org/archives/2004_club.aspx',
    2005: 'https://archive.usaultimate.org/archives/2005_club.aspx',
    2006: 'https://archive.usaultimate.org/archives/2006_club.aspx',
    2007: 'https://archive.usaultimate.org/archives/2007_club.aspx',
    2008: 'https://archive.usaultimate.org/archives/2008_club.aspx',
    2009: 'https://archive.usaultimate.org/archives/2009_club.aspx',
    2010: 'https://archive.usaultimate.org/archives/2010_club.aspx',
    2011: 'https://archive.usaultimate.org/archives/2011_club.aspx',
    2012: 'https://archive.usaultimate.org/archives/2012_club.aspx',
    2013: 'https://archive.usaultimate.org/archives/2013_club.aspx',
    2014: 'https://archive.usaultimate.org/archives/2014_club.aspx',
    2015: 'https://archive.usaultimate.org/archives/2015_club.aspx',
    2016: 'https://archive.usaultimate.org/archives/2016_club.aspx',
    2017: 'https://archive.usaultimate.org/archives/2017_club.aspx',
    2018: 'https://archive.usaultimate.org/archives/2018_club.aspx',
    2019: 'https://archive.usaultimate.org/archives/2019_club.aspx'
}

In [92]:
all_standings = scrape_multiple_urls_dict(urls)

Scraping data for the year 1979 from https://archive.usaultimate.org/archives/1979_club.aspx...
Scraping data for the year 1980 from https://archive.usaultimate.org/archives/1980_club.aspx...
Scraping data for the year 1981 from https://archive.usaultimate.org/archives/1981_club.aspx...
Scraping data for the year 1982 from https://archive.usaultimate.org/archives/1982_club.aspx...
Scraping data for the year 1983 from https://archive.usaultimate.org/archives/1983_club.aspx...
Scraping data for the year 1984 from https://archive.usaultimate.org/archives/1984_club.aspx...
Scraping data for the year 1985 from https://archive.usaultimate.org/archives/1985_club.aspx...
Scraping data for the year 1986 from https://archive.usaultimate.org/archives/1986_club.aspx...
Scraping data for the year 1987 from https://archive.usaultimate.org/archives/1987_club.aspx...
Scraping data for the year 1988 from https://archive.usaultimate.org/archives/1988_club.aspx...
Scraping data for the year 1989 from htt

In [93]:
for url, standings in all_standings.items():
    print(f"\nStandings for {url}:")
    print(standings)


Standings for 1979:
['1 – Glassboro', '2 – Condors', '3 – Michigan State', '4 – Orlando', '5 – Cornell']

Standings for 1980:
['1 – Glassboro', '2 – Aerodisc', '3 – Michigan State', '4 – Condors', '5 – Sky Pilots']

Standings for 1981:
['1 – Condors', '2 – Knights of Nee', '3 – Sky Pilots', '4 – Hostages', '5 – Michigan State']

Standings for 1982:
['1 – Rude Boys', '2 – Tunas', '3 – Gang', '4 – Hostages', '5 – Windy City', '6 – Sky Pilots', '7 – Chain Lightning', '8 – Glassboro', '9 – Fun Hogs', '10 – Flying Circus']

Standings for 1983:
['1 – Windy City', '2 – Spinoffs', '3T – Sky Pilots', '3T – Condors', '5T – Tunas', '5T – Rude Boys', '7T – The Gang', '7T – Static Disc', '9T – Kaboom', '9T – Fat Women']

Standings for 1984:
['1 – Tunas', '2 – Flying Circus', '3T – Windy City', '3T – Condors', '5T – Kaboom', '5T – Rude Boys', '7T – Miami Refugees', '7T – Fat Women', '9T – Static Disc', '9T – The Gang']

Standings for 1985:
['1 – Flying Circus', '2 – Kaboom', '3T – Windy City', '3T 

In [94]:
# # Fix Pittsburgh entry in 2022 standings
# if 2022 in all_standings:
#     all_standings[2022] = [
#         entry.replace('T3 –Pittsburgh', 'T3 – Pittsburgh')  # Note the en dash (–) not hyphen (-)
#         for entry in all_standings[2022]
#     ]

# # Verify the fix
# print("2022 standings after fix:")
# for entry in all_standings[2022]:
#     print(entry)

In [95]:
# Create empty lists to store our data
data = []

# Iterate through the standings dictionary
for year, standings in all_standings.items():
    if isinstance(standings, list):  # Only process if we have actual standings data
        # Counter for actual finish position
        position_counter = 1
        
        for entry in standings:
            # Split on the dash, handling both formats
            if ' – ' in entry:  # en dash
                rank, team = entry.split(' – ', 1)
            elif ' - ' in entry:  # regular hyphen
                rank, team = entry.split(' - ', 1)
            else:
                print(f"Warning: No dash found in {entry} for year {year}")
                continue
                
            # Clean up the rank and team
            rank = rank.strip()
            team = team.strip()
            
            # Store the actual finish position
            list_finish = position_counter
            position_counter += 1
            
            data.append({
                'Year': year,
                'Team': team,
                'Web_Finish': rank,  # Original format (e.g., "T3", "1")
                'List_Finish': list_finish  # Sequential position (1, 2, 3, etc.)
            })

# Create DataFrame
df = pd.DataFrame(data)

In [96]:
# Export to CSV
df.to_csv('club-mens-raw-rankings.csv', index=False)
print("Data exported to 'club-mens-raw-rankings.csv'")

Data exported to 'club-mens-raw-rankings.csv'


In [97]:
# Get unique team names and sort alphabetically
unique_teams_raw = sorted(df['Team'].unique())

# Print the teams
print("All raw teams in alphabetical order:")
for team in unique_teams_raw:
    print(team)

All raw teams in alphabetical order:
Aerodisc
Amanata
Anodyne
Arm & Hammer
BAT
Big Ass Truck
Big Brother
Big River
Blackjack
Blaze of Glory
Bodhi
Bohdi
Bomb Squad
Bonzi
Boost Mobile
Booty Quake
Boss Hogg
Boston Ultimate
Boulder Beer
Boulder Stains
COGZ
Cbass
Chain Lightning
Chesapeake
Cojones
Commonwealth
Condors
Cornell
Death or Glory
DiG
Dig
Double Happiness
Doublewide
Drivers
Earth Atomizer
East Bay
El Diablo
Electric Pig
FBI
Fat Women
First Time Gary
Florida
Florida United
Flying Circus
Fun Hogs
Furious George
G-Unit
GOAT
Gang
Glassboro
Graffiti
Grafitti
H.I.P.
High Five
Hostages
Houndz
Huckin' Foosiers
Iguanas
Ironside
JAM
Jaga
Jam
Johnny Bravo
Kaboom
Kaos
Knights of Nee
Lemon
Looney Tunes
Los Guapos
Machine
Madcow
Madison
Madison Club
Metal
Miami
Miami Refugees
Michigan State
Monster
Mr. Bubble
Mr. Pouce
New York
Nice Guys
Night Train
Oakland
Oregon Donors
Orlando
PBR Streetgang
Parking Lot JAM
Patrol
Philmore
Pike
PoNY
Polo Club
Port City Slickers
Potomac
Prairie Fire
Pump House

In [98]:
def clean_team_name(name):
    # First, strip whitespace and convert to title case
    name = name.strip().title()
    
    # Standardize team names
    replacements = {
        "Bohdi": "Bodhi",
        "Dig": "DiG",
        "Grafitti": "Graffiti",
        "Jam": "JAM",
        "Gang": "The Gang",
        "Miami Refugees": "Refugees",
        "PBR Streetgang": "Streetgang",
        "Rhino": "Rhino Slam!",
        "BAT": "Big Ass Truck",
    }
    
    for old, new in replacements.items():
        if name.upper() == old.upper():  # Case-insensitive replacement
            name = new
    
    return name

In [99]:
# Apply the cleaning function to create new column
df['Team_Clean'] = df['Team'].apply(clean_team_name)

# Print unique teams after cleaning to verify changes
print("All teams after cleaning (alphabetically):")
for team in sorted(df['Team_Clean'].unique()):
    print(team)

All teams after cleaning (alphabetically):
Aerodisc
Amanata
Anodyne
Arm & Hammer
Big Ass Truck
Big Brother
Big River
Blackjack
Blaze Of Glory
Bodhi
Bomb Squad
Bonzi
Boost Mobile
Booty Quake
Boss Hogg
Boston Ultimate
Boulder Beer
Boulder Stains
Cbass
Chain Lightning
Chesapeake
Cogz
Cojones
Commonwealth
Condors
Cornell
Death Or Glory
DiG
Double Happiness
Doublewide
Drivers
Earth Atomizer
East Bay
El Diablo
Electric Pig
Fat Women
Fbi
First Time Gary
Florida
Florida United
Flying Circus
Fun Hogs
Furious George
G-Unit
Glassboro
Goat
Graffiti
H.I.P.
High Five
Hostages
Houndz
Huckin' Foosiers
Iguanas
Ironside
JAM
Jaga
Johnny Bravo
Kaboom
Kaos
Knights Of Nee
Lemon
Looney Tunes
Los Guapos
Machine
Madcow
Madison
Madison Club
Metal
Miami
Michigan State
Monster
Mr. Bubble
Mr. Pouce
New York
Nice Guys
Night Train
Oakland
Oregon Donors
Orlando
Parking Lot Jam
Patrol
Philmore
Pike
Polo Club
Pony
Port City Slickers
Potomac
Prairie Fire
Pump House 5
R & B
Rage
Randall'S Island
Red Tide
Refugees
Revolve

In [100]:
df

,Year,Team,Web_Finish,List_Finish,Team_Clean
0,1979,Glassboro,1,1,Glassboro
1,1979,Condors,2,2,Condors
2,1979,Michigan State,3,3,Michigan State
3,1979,Orlando,4,4,Orlando
4,1979,Cornell,5,5,Cornell
...,...,...,...,...,...
546,2019,Temper,12,12,Temper
547,2019,GOAT,13,13,Goat
548,2019,DiG,14,14,DiG
549,2019,Johnny Bravo,15,15,Johnny Bravo


In [101]:
def transform_rank_format(df, column_name):
    def convert_format(rank):
        # If it's already in T# format, keep it
        if isinstance(rank, str) and rank.startswith('T'):
            return rank
        
        # If it ends with T, convert to T# format
        if isinstance(rank, str) and rank.endswith('T'):
            number = rank[:-1]  # Remove the T
            return f'T{number}'
        
        return rank  # Return unchanged if no T

    # Apply the conversion to the specified column
    df[column_name] = df[column_name].apply(convert_format)
    
    return df

In [102]:
# Create new dataframe without URL and Team columns
df_T = df.drop(['Team'], axis=1)

# Apply the conversion to the specified column
df = transform_rank_format(df_T, 'Web_Finish')

# Rename Team_Clean to Team for clarity
df_T = df_T.rename(columns={'Team_Clean': 'Team', 'List_Finish': 'Rank', 'Web_Finish': 'T_Rank'})

# Reorder columns
df_T = df_T[['Team', 'Year', 'Rank', 'T_Rank']]

# Manually change a rank value for UNC Wilmington in 2021
# df_T.loc[(df_T['Team'] == 'UNC Wilmington') & (df_T['Year'] == 2021), 'T_Rank'] = 'T9'

# Verify the first few rows to check the mapping
df_T

# Export to CSV
df_T.to_csv('club-mens-rankings.csv', index=False)
print("Data exported to 'club-mens-rankings.csv'")

Data exported to 'club-mens-rankings.csv'


In [103]:
# # Read the CSV
# df_with_less_columns_for_tableau = pd.read_csv('ultimate_standings_clean.csv')

# # Group by year and check rank sequences
# found_issues = False
# for year in sorted(df_with_less_columns_for_tableau['Year'].unique()):
#     year_data = df_with_less_columns_for_tableau[df_with_less_columns_for_tableau['Year'] == year].sort_values('Rank')
#     ranks = year_data['Rank'].tolist()
#     expected_ranks = list(range(1, len(ranks) + 1))
    
#     if ranks != expected_ranks:
#         found_issues = True
#         print(f"\nYear {year} has non-sequential ranks:")
#         print("Expected:", expected_ranks)
#         print("Actual:", ranks)
#         print("\nFull data for this year:")
#         print(year_data[['Team', 'Rank']].to_string())

# if not found_issues:
#     print("All years have sequential ranks without gaps! ✓")

In [104]:
df

,Year,Web_Finish,List_Finish,Team_Clean
0,1979,1,1,Glassboro
1,1979,2,2,Condors
2,1979,3,3,Michigan State
3,1979,4,4,Orlando
4,1979,5,5,Cornell
...,...,...,...,...
546,2019,12,12,Temper
547,2019,13,13,Goat
548,2019,14,14,DiG
549,2019,15,15,Johnny Bravo


In [109]:
# Read in the existing CSV
df_T = pd.read_csv('club-mens-rankings.csv')

# Remove any existing 2021-2024 entries
df_T = df_T[~df_T['Year'].isin([2021, 2022, 2023, 2024])]

# Create 2021 data
data_2021 = {
    'Team': [
        'Ring of Fire', 'PoNY', 'Sockeye', 'Rhino Slam!', 'Machine',
        'Truck Stop', 'DiG', 'Temper', 'Johnny Bravo', 'Chain Lightning',
        'Condors', 'Sprout', 'Revolver', 'Sub Zero', 'Killjoys',
        'Lotus'
    ],
    'Year': [2021] * 16,
    'Rank': list(range(1, 17)),
    'T_Rank': [
        '1', '2', 'T3', 'T3', '5', 
        '6', 'T7', 'T7', 'T9', 'T9',
        '11', '12', '13', '14', '15',
        '16'
    ]
}

# Create 2022 data
data_2022 = {
    'Team': [
        'Johnny Bravo', 'Truck Stop', 'PoNY', 'Rhino Slam!', 'Machine',
        'Ring of Fire', 'Doublewide', 'Revolver', 'Vault', 'Chain Lightning',
        'Temper', 'Omen', 'GOAT', 'Sockeye', 'Mad Men',
        'Condors'
    ],
    'Year': [2022] * 16,
    'Rank': list(range(1, 17)),
    'T_Rank': [
        '1', '2', 'T3', 'T3', '5', 
        '6', 'T7', 'T7', 'T9', 'T9',
        '11', '12', '13', '14', '15',
        '16'
    ]
}

# Create 2023 data
data_2023 = {
    'Team': [
        'Truck Stop', 'Machine', 'Johnny Bravo', 'Ring of Fire', 'DiG',
        'PoNY', 'Rhino Slam!', 'Revolver', 'Furious George', 'Chain Lightning',
        'Doublewide', 'Raleigh-Durham United', 'Sub Zero', 'Vault', 'Dark Star',
        'Blueprint'
    ],
    'Year': [2023] * 16,
    'Rank': list(range(1, 17)),
    'T_Rank': [
        '1', '2', 'T3', 'T3', '5', 
        '6', 'T7', 'T7', 'T9', 'T9',
        '11', '12', '13', '14', '15',
        '16'
    ]
}

# Create 2024 data
data_2024 = {
    'Team': [
        'Rhino Slam!', 'PoNY', 'Revolver', 'Machine', 'DiG',
        'Truck Stop', 'Ring of Fire', 'Johnny Bravo', 'Chain Lightning', 'Furious George',
        'Doublewide', 'Mallard', 'Sockeye', 'Raleigh-Durham United', 'GOAT',
        'Shrimp'
    ],
    'Year': [2024] * 16,
    'Rank': list(range(1, 17)),
    'T_Rank': [
        '1', '2', 'T3', 'T3', '5', 
        '6', 'T7', 'T7', 'T9', 'T9',
        '11', '12', '13', '14', '15',
        '16'
    ]
}

# Convert to DataFrames
new_df_2021 = pd.DataFrame(data_2021)
new_df_2022 = pd.DataFrame(data_2022)
new_df_2023 = pd.DataFrame(data_2023)
new_df_2024 = pd.DataFrame(data_2024)

# Append to existing DataFrame
df_T = pd.concat([df_T, new_df_2021, new_df_2022, new_df_2023, new_df_2024], ignore_index=True)

# Export to CSV
df_T.to_csv('club-mens-rankings.csv', index=False)